In [1]:
import requests
import xml.etree.ElementTree as ET
import pandas as pd
from requests.utils import unquote
import re

/Users/ijongseung/house_electric/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
def get_kapt_energyconsumption(kaptcode, date):
    service_key = "iCNxo2r0TdZnnV63%2FItO%2BQrOUqJakXCxx%2Fm20BsCp53DGZzJMDd1%2F7jOGLYQE%2BSn%2B1EQeSeIhUsTIyQ5dYgy4Q%3D%3D"
    
    url = "http://apis.data.go.kr/1611000/ApHusEnergyUseInfoOfferService"  # 엔드포인트 확인 필요

    params = {
        "serviceKey": service_key,
        "kaptCode": kaptcode,
        "reqDate": date,
        "_type": "xml"
    }

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36",
        "Accept": "application/xml"
    }

    response = requests.get(url, params=params, headers=headers)
    
    if response.status_code == 200:
        root = ET.fromstring(response.text)
        
        # 응답 출력하여 확인
        print(response.text)

        data_list = []
        for item in root.findall(".//item"):
            data_dict = {child.tag: child.text if child.text else 'N/A' for child in item}
            data_list.append(data_dict)
        
        df = pd.DataFrame(data_list)
        return df
    else:
        print(f"오류 발생: HTTP {response.status_code}")
        return None

# 테스트
df = get_kapt_energyconsumption("A10027953", "202201")
print(df)


오류 발생: HTTP 500
None


In [3]:
import requests
import xml.etree.ElementTree as ET
import pandas as pd
from urllib.parse import unquote

def get_kapt_energyconsumption(kaptcode, date):
    """단일 아파트 코드(kaptcode)와 날짜(date)에 대해 에너지 소비량을 조회"""
    
    # 공공데이터포털 서비스 키 (디코딩)
    service_key = unquote("iCNxo2r0TdZnnV63%2FItO%2BQrOUqJakXCxx%2Fm20BsCp53DGZzJMDd1%2F7jOGLYQE%2BSn%2B1EQeSeIhUsTIyQ5dYgy4Q%3D%3D")
    
    # API 엔드포인트
    url = "https://apis.data.go.kr/1611000/ApHusEnergyUseInfoOfferService/getHsmpApHusUsgQtyInfoSearch"

    # 요청 파라미터 설정
    params = {
        "serviceKey": service_key,  
        "kaptCode": kaptcode,       
        "reqDate": date,            
        "_type": "xml"              
    }

    # 요청 헤더 설정
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36",
        "Accept": "application/xml"
    }

    # API 요청
    response = requests.get(url, params=params, headers=headers)

    # 응답 상태 코드 확인
    if response.status_code == 200:
        root = ET.fromstring(response.text)
        data_list = []

        # XML에서 item 태그 찾기
        for item in root.findall(".//item"):
            data_dict = {child.tag: child.text if child.text else 'N/A' for child in item}
            data_list.append(data_dict)

        # 데이터가 없는 경우 확인 메시지 출력
        if not data_list:
            print(f"데이터 없음: kaptCode={kaptcode}, date={date}")

        # 데이터프레임 변환
        return pd.DataFrame(data_list)

    else:
        print(f"오류 발생: HTTP {response.status_code} (kaptCode={kaptcode}, date={date})")
        print("응답 내용:", response.text)  
        return None

def get_multiple_kapt_energyconsumption(kaptcodes, dates):
    """여러 개의 아파트 코드(kaptcodes)와 날짜(dates)에 대해 에너지 소비량을 조회"""
    
    all_results = []  # 모든 데이터를 저장할 리스트

    for kaptcode in kaptcodes:
        for date in dates:
            df = get_kapt_energyconsumption(kaptcode, date)
            if df is not None and not df.empty:
                df["kaptCode"] = kaptcode  # 아파트 코드 추가
                df["reqDate"] = date  # 요청 날짜 추가
                all_results.append(df)

    # 모든 데이터를 하나의 데이터프레임으로 합치기
    if all_results:
        final_df = pd.concat(all_results, ignore_index=True)
        final_df.rename(columns = {
                "elect" : "전기사용금액",
                "gas" : "가스사용금액",
                "heat" : "난방사용금액",
                "helect" : "전기사용량",
                "hgas" : "가스사용량",
                "hheat" : "난방사용량",
                "hwaterCool" : "냉수사용량",
                "hwaterHot" : "온수사용량",
                # "kaptCode" : "아파트코드",
                "waterCool" : "냉수사용금액",
                "waterHot" : "온수사용금액",
                "reqDate" : "집계날짜"}, inplace=True)
        return final_df
    
    else:
        print("❗ 모든 요청에서 데이터를 가져오지 못했습니다.")
        return pd.DataFrame()

# 테스트 실행 
df = pd.read_csv("/Users/ijongseung/house_electric/address_data/seoul_data.csv") # 

kaptcodes = df["kaptCode"].tolist()  # 여러 개의 아파트 코드
dates = ["202401"]  # 일단 가장 최근 날짜로 변형한다.

energy_data = get_multiple_kapt_energyconsumption(kaptcodes, dates)

In [8]:
# 에너지 데이터 저장
energy_data.to_csv("/Users/ijongseung/house_electric/address_data/energy_data.csv", index=False)

### 서울시 데이터와 에너지 사용량 매칭

In [22]:
# 테스트 실행 
df = pd.read_csv("/Users/ijongseung/house_electric/address_data/seoul_data.csv") # 
df_energy_consumption = pd.read_csv("/Users/ijongseung/house_electric/address_data/energy_data.csv") # /Users/ijongseung/house_electric/address_data/energy_data.csv

# 데이터 묶기
df_merged = pd.merge(df, df_energy_consumption, on="kaptCode", how="outer")

In [32]:
# 송파구 데이터만 수집
df_gangnam = df_merged[df_merged["kaptAddr"].str.contains("종로구")]
df_gangnam.rename(columns = {"doroJuso": "address"}, inplace=True)

/var/folders/7n/zyzjp7wj0cn68mx_s16mw1m00000gn/T/ipykernel_80799/1892025017.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_gangnam.rename(columns = {"doroJuso": "address"}, inplace=True)


In [33]:
df_gangnam

,bjdCode,codeAptNm,codeHallNm,codeHeatNm,codeMgrNm,codeSaleNm,address,hoCnt,kaptAcompany,kaptAddr,...,가스사용금액,난방사용금액,전기사용량,가스사용량,난방사용량,냉수사용량,온수사용량,냉수사용금액,온수사용금액,집계날짜
41,1171010600,아파트,혼합식,개별난방,위탁관리,분양,NaN,1.0,현대산업개발,서울특별시 송파구 삼전동 39- 삼전현대아파트,...,0,0,0,0,0,0,0,0,0,202401
51,1171011400,아파트,혼합식,개별난방,자치관리,분양,NaN,107.0,(주)건원건설,서울특별시 송파구 마천동 21- 송파건원여미지아파트,...,0,0,0,0,0,0,0,0,0,202401
54,1171010700,아파트,복도식,개별난방,위탁관리,분양,NaN,105.0,조합,서울특별시 송파구 가락동 142- 가락풍림아파트,...,0,0,0,0,0,0,0,0,0,202401
75,1171010700,주상복합,복도식,중앙난방,위탁관리,분양,NaN,113.0,주식회사 로쿠스,서울특별시 송파구 가락동 98- 송파가락두산위브센티움,...,0,0,0,0,0,0,0,0,0,202401
78,1171011300,아파트,계단식,개별난방,위탁관리,분양,NaN,140.0,어울림거여동아파트관리사무소,서울특별시 송파구 거여동 593- 어울림거여동아파트,...,0,0,0,0,0,0,0,0,0,202401
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2165,1171010300,아파트,계단식,개별난방,위탁관리,분양,서울특별시 송파구 풍성로26길 31,166.0,우신.재경연립주택및강동연립주택 재건축정비사업조합,서울특별시 송파구 풍납동 511- 송파현대힐스테이트,...,0,0,60574,0,0,3253,0,4291100,0,202401
2166,1171010700,주상복합,계단식,개별난방,위탁관리,분양,서울특별시 송파구 송파대로28길 27,324.0,성원건설,서울특별시 송파구 가락동 80- 송파성원상떼빌,...,0,0,117925,0,0,4819,0,5477360,0,202401
2167,1171010700,아파트,계단식,개별난방,위탁관리,분양,서울특별시 송파구 송이로19길 15,206.0,쌍용건설,서울특별시 송파구 가락동 52- 가락3차쌍용스윗닷홈(1~4동),...,0,0,91951,0,0,7976,0,4856740,0,202401
2168,1171010700,주상복합,복도식,개별난방,위탁관리(직영+위탁),분양,서울특별시 송파구 중대로 127,160.0,한화건설,서울특별시 송파구 가락동 78-1 한화오벨리스크,...,0,0,20337,0,0,0,0,0,0,202401


In [34]:
# Kakao API Key 입력
KAKAO_API_KEY = "b83a0138ed0d17b68f0c86fe6808b74d"
YOUR_KAKAO_JS_API_KEY = "41c870c490aebd3810bd1152b41c792e"

def clean_address(address):
    if pd.isna(address):  # NaN 값이 들어오면 그대로 반환
        return address
    return re.sub(r"(\d+-?\d*)\s?.*", r"\1", address).strip()


# 지번주소 → 도로명주소 변환 함수
def get_road_address(jibun_address):
    url = "https://dapi.kakao.com/v2/local/search/address.json"
    headers = {"Authorization": f"KakaoAK {KAKAO_API_KEY}"}
    params = {"query": jibun_address}

    response = requests.get(url, headers=headers, params=params)

    if response.status_code == 200:
        result = response.json()
        if result["documents"]:
            road_address = result["documents"][0].get("road_address", {}).get("address_name")
            if road_address:
                print(f"✅ 도로명 주소 변환 성공: {jibun_address} → {road_address}")
                return road_address
            else:
                print(f"❌ 도로명 주소 변환 실패: {jibun_address} (원본 유지)")
                return jibun_address  # 변환 실패 시 원본 유지
    else:
        print(f"API 요청 실패: {response.status_code}, {response.text}")

    return jibun_address  # 변환 실패 시 원본 유지

# 🔹 주소를 위도, 경도로 변환하는 함수
def get_lat_lon(address):
    url = "https://dapi.kakao.com/v2/local/search/address.json"
    headers = {"Authorization": f"KakaoAK {KAKAO_API_KEY}"}
    
    params = {"query": address}
    response = requests.get(url, headers=headers, params=params)

    if response.status_code == 200:
        result = response.json()
        if result["documents"]:
            lat = float(result["documents"][0]["y"])
            lon = float(result["documents"][0]["x"])
            return lat, lon
        else:
            print(f"주소 변환 실패: {address}")  
    else:
        print(f"API 요청 실패: {response.status_code}, {response.text}")

    return None, None

# 🔹 주소 클리닝 (숫자 뒤의 아파트 이름 제거, 지번 유지)
df_gangnam["cleaned_address"] = df_gangnam["kaptAddr"].apply(clean_address)

# 🔹 기존에 "address" 열이 존재하는 경우 변환 스킵
if "address" not in df_gangnam.columns:
    df_gangnam["address"] = None  # "address" 열이 없으면 생성

# 🔹 address 열이 None이면 변환 실행, 값이 있으면 기존 값 유지
df_gangnam["address"] = df_gangnam.apply(
    lambda row: get_road_address(row["cleaned_address"]) if pd.isna(row["address"]) else row["address"], axis=1
)

# 🔹 도로명주소 기반으로 위도/경도 변환
df_gangnam[["lat", "lon"]] = df_gangnam["address"].apply(get_lat_lon).apply(pd.Series)

# 🔹 변환되지 않은 데이터 제거
df_gangnam = df_gangnam.dropna(subset=["lat", "lon"])

# 🔹 전기사용량 열 추가 (전기사용량이 없을 경우 0으로 대체)
if "전기사용량" not in df_gangnam.columns:
    df_gangnam["전기사용량"] = 0

# 🔹 HTML 파일 생성
html_template = f"""
<!DOCTYPE html>
<html>
<head>
    <meta charset="utf-8"/>
    <title>Kakao 지도 - 송파구 아파트</title>
    <script type="text/javascript" src="//dapi.kakao.com/v2/maps/sdk.js?appkey=41c870c490aebd3810bd1152b41c792e"></script>
</head>
<body>
    <div id="map" style="width:800px;height:600px;"></div>
    <script>
        var map = new kakao.maps.Map(document.getElementById('map'), {{
            center: new kakao.maps.LatLng({df_gangnam['lat'].mean()}, {df_gangnam['lon'].mean()}), 
            level: 6 
        }});

        var positions = {df_gangnam.to_json(orient='records')};

        positions.forEach(function(position) {{
            var marker = new kakao.maps.Marker({{
                map: map,
                position: new kakao.maps.LatLng(position.lat, position.lon)
            }});
            
            var infowindow = new kakao.maps.InfoWindow({{
                content: '<div style="padding:5px;"><b>주소:</b> ' + position.address + '<br>' +
                         '<b>전기사용량:</b> ' + position.전기사용량 + ' kWh</div>'
            }});

            kakao.maps.event.addListener(marker, 'click', function() {{
                infowindow.open(map, marker);
            }});
        }});
    </script>
</body>
</html>
"""

# 🔹 HTML 파일 저장
with open("map.html", "w", encoding="utf-8") as f:
    f.write(html_template)

print("HTML 파일이 생성되었습니다. 'python3 -m http.server 8080'을 실행하고 http://localhost:8080/map.html 에 접속하세요!")

/var/folders/7n/zyzjp7wj0cn68mx_s16mw1m00000gn/T/ipykernel_80799/877177732.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_gangnam["cleaned_address"] = df_gangnam["kaptAddr"].apply(clean_address)


✅ 도로명 주소 변환 성공: 서울특별시 송파구 삼전동 39- → 서울 송파구 백제고분로22길 37
✅ 도로명 주소 변환 성공: 서울특별시 송파구 마천동 21- → 서울 송파구 마천로35길 12
✅ 도로명 주소 변환 성공: 서울특별시 송파구 가락동 142- → 서울 송파구 동남로9길 39
✅ 도로명 주소 변환 성공: 서울특별시 송파구 가락동 98- → 서울 송파구 송파대로30길 16
✅ 도로명 주소 변환 성공: 서울특별시 송파구 거여동 593- → 서울 송파구 오금로56길 28
✅ 도로명 주소 변환 성공: 서울특별시 송파구 오금동 29- → 서울 송파구 양재대로72길 42
✅ 도로명 주소 변환 성공: 서울특별시 송파구 방이동 157- → 서울 송파구 위례성대로8길 28
❌ 도로명 주소 변환 실패: 서울특별시 송파구 송파동 245- (원본 유지)
✅ 도로명 주소 변환 성공: 서울특별시 송파구 풍납동 251-2 → 서울 송파구 풍성로14길 19
✅ 도로명 주소 변환 성공: 서울특별시 송파구 방이동 51-4 → 서울 송파구 백제고분로 497
✅ 도로명 주소 변환 성공: 서울특별시 송파구 문정동 104- → 서울 송파구 송이로34길 26
✅ 도로명 주소 변환 성공: 서울특별시 송파구 거여동 136- → 서울 송파구 거마로9길 19
✅ 도로명 주소 변환 성공: 서울특별시 송파구 마천동 9- → 서울 송파구 마천로35길 18
✅ 도로명 주소 변환 성공: 서울특별시 송파구 문정동 136- → 서울 송파구 송이로31길 20


/var/folders/7n/zyzjp7wj0cn68mx_s16mw1m00000gn/T/ipykernel_80799/877177732.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_gangnam["address"] = df_gangnam.apply(


HTML 파일이 생성되었습니다. 'python3 -m http.server 8080'을 실행하고 http://localhost:8080/map.html 에 접속하세요!


/var/folders/7n/zyzjp7wj0cn68mx_s16mw1m00000gn/T/ipykernel_80799/877177732.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_gangnam[["lat", "lon"]] = df_gangnam["address"].apply(get_lat_lon).apply(pd.Series)
/var/folders/7n/zyzjp7wj0cn68mx_s16mw1m00000gn/T/ipykernel_80799/877177732.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_gangnam[["lat", "lon"]] = df_gangnam["address"].apply(get_lat_lon).apply(pd.Series)


### 열 처리 방법

###
df_songpa.rename(columns={
    'kaptName': '단지명',
    'kaptAddr': '법정동주소',
    'codeSaleNm': '분양형태',
    'codeHeatNm': '난방방식',
    'kaptTarea': '건축물대장상 연면적',
    'kaptDongCnt': '동수',
    'hoCnt': '세대수',
    'kaptAcompany': '시공사',
    'kaptBcompany': '시행사',
    'kaptTel': '관리사무소연락처',
    'kaptFax': '관리사무소팩스',
    'kaptUrl': '홈페이지주소',
    'codeAptNm': '단지분류',
    'address': '도로명주소',
    'kaptdaCnt': '호수',
    'codeMgrNm': '관리방식',
    'codeHallNm': '복도유형',
    'kaptUsedate': '사용승인일',
    'kaptMarea': '관리비부과면적',
    'kaptMparea_60': '전용면적별 세대현황_60㎡이하',
    'kaptMparea_85': '전용면적별 세대현황_85㎡이하',
    'kaptMparea_135': '전용면적별 세대현황_135㎡이상',
    'kaptMparea_136': '전용면적별 세대현황_136㎡이상',
    'privArea': '단지 전용면적합',
    'bjdCode': '법정동코드'
}, inplace=True)

In [64]:
# 송파구 데이터만 수집
df_gangnam = df_merged[df_merged["kaptAddr"].str.contains("영등포")]
df_gangnam.rename(columns = {"doroJuso": "address"}, inplace=True)

/var/folders/7n/zyzjp7wj0cn68mx_s16mw1m00000gn/T/ipykernel_80799/1598345245.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_gangnam.rename(columns = {"doroJuso": "address"}, inplace=True)


In [ ]:
import json
import pandas as pd
import requests
import re
# b83a0138ed0d17b68f0c86fe6808b74d"
# "41c870c490aebd3810bd1152b41c792e"
class get_geo_data:
    def __init__(self, KAKAO_API_KEY, KAKAO_JS_API_KEY, district_name):
        self.KAKAO_API_KEY = KAKAO_API_KEY
        self.KAKAO_JS_API_KEY = KAKAO_JS_API_KEY
        self.district_name = district_name

    def get_district_boundary(self, district_name):
        
        with open("line_data.json", "r", encoding="utf-8") as f:
            districts_data = json.load(f)
        
        for district in districts_data["features"]:  # 'features' 리스트를 순회해야 함
            if district["properties"]["name"] == district_name:
                return district["geometry"]["coordinates"][0]  # 폴리곤 좌표 반환
        return []

    def clean_address(self, address):
        if pd.isna(address):  # NaN 값 그대로 유지
            return address
        return re.sub(r"(\d+-?\d*)\s?.*", r"\1", address).strip()

    # 지번주소 → 도로명주소 변환 함수
    def get_road_address(self, jibun_address):
        url = "https://dapi.kakao.com/v2/local/search/address.json"
        headers = {"Authorization": f"KakaoAK {self.KAKAO_API_KEY}"}
        params = {"query": jibun_address}

        response = requests.get(url, headers=headers, params=params)

        if response.status_code == 200:
            result = response.json()
            if result["documents"]:
                road_address = result["documents"][0].get("road_address", {}).get("address_name")
                if road_address:
                    print(f"✅ 도로명 주소 변환 성공: {jibun_address} → {road_address}")
                    return road_address
                else:
                    print(f"❌ 도로명 주소 변환 실패: {jibun_address} (원본 유지)")
                    return jibun_address  # 변환 실패 시 원본 유지
        else:
            print(f"API 요청 실패: {response.status_code}, {response.text}")     
                
        return jibun_address  # API 요청 실패 시 원본 유지

    # 주소 → 위도/경도 변환 함수
    def get_lat_lon(self, address):
        url = "https://dapi.kakao.com/v2/local/search/address.json"
        headers = {"Authorization": f"KakaoAK {self.KAKAO_API_KEY}"}
    
        params = {"query": address}
        response = requests.get(url, headers=headers, params=params)

        if response.status_code == 200:
            result = response.json()
            if result["documents"]:
                lat = float(result["documents"][0]["y"])
                lon = float(result["documents"][0]["x"])
                return lat, lon
            
        return None, None

    def preprocessing(self, df):
        df_revision = df[df["kaptAddr"].str.contains(self.district_name)]
        df_revision.rename(columns = {"doroJuso": "address"}, inplace=True)
        
        # 주소 클리닝 (숫자 뒤의 아파트 이름 제거, 지번 유지)
        df_revision["cleaned_address"] = df_revision["kaptAddr"].apply(clean_address)

        # 도로명 주소 변환 (기존 "address" 값이 없으면 변환 실행)
        df_revision["address"] = df_revision.apply(
        lambda row: get_road_address(row["cleaned_address"]) if pd.isna(row["address"]) else row["address"], axis=1
        )

        # 도로명주소 기반 위도/경도 변환
        df_revision[["lat", "lon"]] = df_revision["address"].apply(get_lat_lon).apply(pd.Series)

        # 변환되지 않은 데이터 제거
        df_filtered = df_revision.dropna(subset=["lat", "lon"])

        # 전기사용량 열 추가 (전기사용량이 없을 경우 0으로 대체)
        if "전기사용량" not in df_filtered.columns:
            df_filtered["전기사용량"] = 0

        return df_filtered
    
    def generate_kakao_map_html(self):
        district_boundary = self.get_district_boundary(self.district_name)
        df_filtered = self.preprocessing(df)
        html_template = f"""
    
        <!DOCTYPE html>
        <html>
        <head>
            <meta charset="utf-8"/>
                <title>Kakao 지도 - {self.district_name}</title>
    <script type="text/javascript" src="//dapi.kakao.com/v2/maps/sdk.js?appkey={self.KAKAO_JS_API_KEY}"></script>
</head>
<body>
    <div id="map" style="width:100%;height:600px;"></div>
    <script>
        var map = new kakao.maps.Map(document.getElementById('map'), {{
            center: new kakao.maps.LatLng({df_filtered['lat'].mean()}, {df_filtered['lon'].mean()}), 
            level: 6 
        }});

        // ✅ 선택한 구(예: 종로구) 행정구역 경계
        var districtBoundary = {json.dumps(district_boundary)};

        var polygonPath = districtBoundary.map(coord => new kakao.maps.LatLng(coord[1], coord[0]));

        var polygon = new kakao.maps.Polygon({{
            path: polygonPath,
            strokeWeight: 3,
            strokeColor: '#FF0000',
            strokeOpacity: 0.8,
            fillColor: '#FF0000',
            fillOpacity: 0.3
        }});

        polygon.setMap(map);

        // ✅ 선택한 구 내 아파트 마커 표시
        var positions = {df_filtered.to_json(orient='records')};

        positions.forEach(function(position) {{
            var marker = new kakao.maps.Marker({{
                map: map,
                position: new kakao.maps.LatLng(position.lat, position.lon)
            }});
            
            var infowindow = new kakao.maps.InfoWindow({{
                content: '<div style="padding:5px;"><b>주소:</b> ' + position.address + '<br>' +
                         '<b>전기사용량:</b> ' + position.전기사용량 + ' kWh</div>'
            }});

            kakao.maps.event.addListener(marker, 'click', function() {{
                infowindow.open(map, marker);
                    }});
                }});
            </script>
        </body>
    </html> """
        
        # ✅ HTML 파일 저장
        with open("map.html", "w", encoding="utf-8") as f:
            f.write(html_template)
    
        print(f"✅ '{self.district_name}' 구역의 지도 HTML 파일이 생성되었습니다. http://localhost:8080/map.html")


✅ '강서구'의 경계 데이터를 로드했습니다.


/var/folders/7n/zyzjp7wj0cn68mx_s16mw1m00000gn/T/ipykernel_80799/2279620244.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_gangnam["cleaned_address"] = df_gangnam["kaptAddr"].apply(clean_address)


✅ 도로명 주소 변환 성공: 서울특별시 강서구 방화동 850- → 서울 강서구 개화동로21길 22
✅ 도로명 주소 변환 성공: 서울특별시 강서구 등촌동 711- → 서울 강서구 양천로 420
✅ 도로명 주소 변환 성공: 서울특별시 강서구 화곡동 354-54 → 서울 강서구 강서로17라길 3
✅ 도로명 주소 변환 성공: 서울특별시 강서구 화곡동 1011-6 → 서울 강서구 강서로45다길 30-17
✅ 도로명 주소 변환 성공: 서울특별시 강서구 방화동 906- → 서울 강서구 양천로24길 13
✅ 도로명 주소 변환 성공: 서울특별시 강서구 가양동 448-9 → 서울 강서구 양천로 521
✅ 도로명 주소 변환 성공: 서울특별시 강서구 화곡동 1130-7 → 서울 강서구 화곡로58길 92
✅ 도로명 주소 변환 성공: 서울특별시 강서구 가양동 1462- → 서울 강서구 허준로 16-9
✅ 도로명 주소 변환 성공: 서울특별시 강서구 마곡동 1577- → 서울 강서구 양천로30길 108
✅ 도로명 주소 변환 성공: 서울특별시 강서구 내발산동 745- → 서울 강서구 수명로2길 108
✅ 도로명 주소 변환 성공: 서울특별시 강서구 염창동 249-11 → 서울 강서구 양천로73길 6
✅ 도로명 주소 변환 성공: 서울특별시 강서구 방화동 893- → 서울 강서구 방화대로41길 6
✅ 도로명 주소 변환 성공: 서울특별시 강서구 염창동 264-27 → 서울 강서구 양천로 686-5
✅ 도로명 주소 변환 성공: 서울특별시 강서구 화곡동 1175- → 서울 강서구 가로공원로80길 40
✅ 도로명 주소 변환 성공: 서울특별시 강서구 염창동 262-2 → 서울 강서구 공항대로75길 42
✅ 도로명 주소 변환 성공: 서울특별시 강서구 염창동 303- → 서울 강서구 양천로71길 54
❌ 도로명 주소 변환 실패: 서울특별시 강서구 공항동 11-10 (원본 유지)
✅ 도로명 주소 변환 성공: 서울특별시 강서구 등촌동 640- → 서울 강서구 화곡로 390
✅ 도로명 주소 변환 성공: 서울특별시

/var/folders/7n/zyzjp7wj0cn68mx_s16mw1m00000gn/T/ipykernel_80799/2279620244.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_gangnam["address"] = df_gangnam.apply(


✅ '강서구' 구역의 지도 HTML 파일이 생성되었습니다. http://localhost:8080/map.html


/var/folders/7n/zyzjp7wj0cn68mx_s16mw1m00000gn/T/ipykernel_80799/2279620244.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_gangnam[["lat", "lon"]] = df_gangnam["address"].apply(get_lat_lon).apply(pd.Series)
/var/folders/7n/zyzjp7wj0cn68mx_s16mw1m00000gn/T/ipykernel_80799/2279620244.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_gangnam[["lat", "lon"]] = df_gangnam["address"].apply(get_lat_lon).apply(pd.Series)


In [ ]:
# 🔹 주소 클리닝 (숫자 뒤의 아파트 이름 제거, 지번 유지)
df_gangnam["cleaned_address"] = df_gangnam["kaptAddr"].apply(clean_address)

# 🔹 도로명 주소 변환 (기존 "address" 값이 없으면 변환 실행)
df_gangnam["address"] = df_gangnam.apply(
    lambda row: get_road_address(row["cleaned_address"]) if pd.isna(row["address"]) else row["address"], axis=1
)

# 🔹 도로명주소 기반 위도/경도 변환
df_gangnam[["lat", "lon"]] = df_gangnam["address"].apply(get_lat_lon).apply(pd.Series)

# 🔹 변환되지 않은 데이터 제거
df_filtered = df_gangnam.dropna(subset=["lat", "lon"])

# 🔹 전기사용량 열 추가 (전기사용량이 없을 경우 0으로 대체)
if "전기사용량" not in df_filtered.columns:
    df_filtered["전기사용량"] = 0